In [18]:
#coding:utf-8
%reload_ext autoreload 
%autoreload 2
import os
import sys

project_basedir = '..'
sys.path.append(project_basedir)
from cchess_zero import cbf
from cchess_zero import board_visualizer
from matplotlib import pyplot as plt
import os
from cchess_zero.gameboard import *
from net import resnet
import common
from gameplays import gameplay,players
from gameplays.analysis import get_km_json
import json
%matplotlib inline

In [19]:
! ls '../data/distributed/' | tail

2018-08-25_22-41-37_810_mcts-mcts_net-net_b.cbf
2018-08-25_22-41-52_626_mcts-mcts_net-net_peace.cbf
2018-08-25_22-42-14_923_mcts-mcts_net-net_w.cbf
2018-08-25_22-42-28_909_mcts-mcts_net-net_w.cbf
2018-08-25_22-42-30_907_mcts-mcts_net-net_b.cbf
2018-08-25_22-42-39_253_mcts-mcts_net-net_w.cbf
2018-08-25_22-42-43_552_mcts-mcts_net-net_w.cbf
2018-08-25_22-43-01_131_mcts-mcts_net-net_w.cbf
2018-08-25_22-43-31_266_mcts-mcts_net-net_w.cbf
2018-08-25_22-44-03_804_mcts-mcts_net-net_b.cbf


In [20]:

fullpath = '../data/distributed/2018-08-25_22-43-01_131_mcts-mcts_net-net_w.cbf'
moves = cbf.cbf2move(fullpath)
fname = fullpath.split('/')[-1]
print(fname)
state = gameplay.GameState()

2018-08-25_22-43-01_131_mcts-mcts_net-net_w.cbf


In [21]:
from PIL import Image
import imageio
imgs_for_gif = []
state = gameplay.GameState()
statestr = 'RNBAKABNR/9/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5c1/9/rnbakabnr'
for i in moves[:]: 
    statestr = GameBoard.sim_do_action(i,statestr)
    imgs_for_gif.append(board_visualizer.get_board_img(statestr))
    state.do_move(i)
imgs_for_gif = imgs_for_gif[:]
imageio.mimsave('../data/gif/{}.gif'.format(fname[:-4]), imgs_for_gif,duration=0.5)

In [22]:
i,len(moves)

('e9f9', 104)

In [23]:
def display_gif(fn):
    from IPython import display
    return display.HTML('<img src="{}">'.format(fn))

In [24]:
display_gif('../data/gif/{}.gif'.format(fname[:-4]))

# ANALYSIS

In [112]:
network = resnet.get_model(os.path.join(project_basedir,
    '../data/prepare_weight/2018-08-17_11-13-25'[3:]),
    common.board.create_uci_labels(),GPU_CORE=[None],FILTERS=128,NUM_RES_LAYERS=7)

0
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
inside res net [None, 10, 9, 128]
INFO:tensorflow:Restoring parameters from ../data/prepare_weight/2018-08-17_11-13-25


In [150]:
network_player = players.NetworkPlayer(state.currentplayer,network,n_playout=400)

In [151]:
network_player.make_move(state=state,actual_move=False)

('c0e2', 0.15995199868339108)

In [152]:
network_player.mcts_policy._root._Q

-0.15995199868339108

In [115]:
childs = network_player.mcts_policy._root._children
for move in childs:
    val = childs[move]
    print(move,val._n_visits,val._P,val._Q)

a3a4 0 3.845509e-05 0
b0a0 0 0.00014557115 0
b0b1 0 0.0014822551 0
b0b2 0 0.00090064644 0
b0b3 0 0.002988217 0
b0b4 0 0.005231117 0
b0b5 0 0.0024759679 0
b0b6 15 0.03290768 0.9476261337598165
b0b7 103 0.217501 0.9991115539976694
b0b8 1 0.012088603 0.9533587098121643
b0b9 0 0.00024749892 0
c2a1 0 4.3057804e-05 0
c2e3 64 0.13091888 0.9910937855020163
c3c4 11 0.023087915 0.9850160587917675
d5b4 0 6.904853e-05 0
d5b6 0 0.0014900416 0
d5e3 0 0.0048407777 0
d5f4 0 0.0009169835 0
d5f6 0 0.0024604101 0
e0f0 0 0.0016041754 0
g4g5 0 0.0006683134 0
g8a8 0 0.0063022277 0
g8b8 0 0.011707161 0
g8c8 13 0.015473258 0.9907070260781509
g8d8 8 0.01288397 0.9813164174556732
g8e8 0 0.0066475626 0
g8f8 18 0.037054785 0.9892583423190646
g8h8 0 0.0070561916 0
g8i8 0 0.00878184 0
g8g5 9 0.013060891 0.9886536002159119
g8g6 14 0.013397759 0.9766642962183271
g8g7 33 0.0649443 0.9887887925812694
g8g9 60 0.12346473 0.9871647278467811
h9e9 0 0.001378259 0
h9g9 0 0.0018126551 0
h9i9 13 0.012728269 0.9928401112556458


In [28]:
datajson = get_km_json(network_player.mcts_policy,state.statestr,depth=40)

In [29]:
with open('../data/mstc_vis/sample.km','w') as whdl:
    json.dump(datajson,whdl)